In [1]:
import os
os.chdir('C://Users//17731//PycharmProjects//transformer_test_github//transformer_test')

In [2]:
from utils import generate_padding_mask, generate_future_mask, combine_padding_mask
import torch

batch_size = 2
seq_length = 5
source = torch.tensor([
    [1, 2, 0, 0, 0],
    [4, 5, 6, 0, 0]
])

target = torch.tensor([
    [1, 2, 4, 0, 0],
    [4, 5, 6, 9, 0]
])




In [18]:
padding_mask = (source == 0).unsqueeze(1).unsqueeze(2)
padding_mask = padding_mask.expand(-1, -1, padding_mask.shape[3], -1)
padding_mask

tensor([[[[False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False,  True,  True,  True]]],


        [[[False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True]]]])

In [5]:
padding_mask = (source == 0).unsqueeze(1).unsqueeze(2)
padding_mask.shape
padding_mask = padding_mask | padding_mask.transpose(-2, -1)
padding_mask.shape

padding_mask2 = (target == 0).unsqueeze(1).unsqueeze(2)
padding_mask2 = padding_mask2 | padding_mask2.transpose(-2, -1)

a = padding_mask2[:, :, 0, :].unsqueeze(3)
a.shape
b = padding_mask[:, :, :, 0].unsqueeze(2)
c = a | b
c


tensor([[[[False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True]]],


        [[[False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False,  True,  True],
          [ True,  True,  True,  True,  True]]]])

In [6]:
size = 5
future_mask = torch.triu(torch.ones(size, size, dtype=torch.bool), diagonal=1)
future_mask

tensor([[False,  True,  True,  True,  True],
        [False, False,  True,  True,  True],
        [False, False, False,  True,  True],
        [False, False, False, False,  True],
        [False, False, False, False, False]])

In [3]:
from models import MachineTranslation
import torch.nn as nn
from models.positional_encoding import PositionalEncoding
d_model = 512
n_head = 8
n_encoder_layers = 6
n_decoder_layers = 6
d_feedforward = 2048
dropout = 0.1
lr = 0.01
batch_size = 32
pad_token = 0
source_vocab_dim = 7
target_vocab_dim = 7

source = torch.tensor([
    [1, 2, 0, 0, 0],
    [4, 5, 6, 0, 0]
])

target = torch.tensor([
    [1, 2, 4, 0, 0],
    [4, 5, 6, 3, 0]
])

# model = MachineTranslation(source_vocab_dim, target_vocab_dim, d_model, n_head, n_encoder_layers, n_decoder_layers, d_feedforward, pad_token,
#                  dropout)
# output = model(source, target)
source_embedding = nn.Embedding(source_vocab_dim, d_model)
em = source_embedding(source)
pe = PositionalEncoding(d_model)
output = pe(em)
output

tensor([[[ 0.2869, -0.8287,  1.6055,  ...,  1.4646,  1.3850,  0.6980],
         [ 0.3066,  0.8184,  1.6755,  ...,  0.4155,  0.7556,  1.0276],
         [-0.9671,  2.1433, -0.6516,  ..., -0.3990,  0.0402,  1.1113],
         [-0.9671,  2.1433, -0.6516,  ..., -0.3990,  0.0402,  1.1113],
         [-0.9671,  2.1433, -0.6516,  ..., -0.3990,  0.0402,  1.1113]],

        [[ 3.0833,  0.9481,  1.0377,  ...,  1.6291, -0.8229,  1.1172],
         [ 0.1246,  1.2816,  2.1835,  ...,  0.1720,  0.9406, -0.1546],
         [ 0.4254,  0.0677, -0.6137,  ...,  1.5625,  0.9353,  0.9112],
         [-0.1256,  1.6836,  0.1703,  ..., -0.3990,  0.0403,  1.1113],
         [-0.1256,  1.6836,  0.1703,  ..., -0.3990,  0.0403,  1.1113]]],
       grad_fn=<AddBackward0>)